In [1]:
!pip install transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 7.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10

In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.0 MB/s eta 0:00:00


In [3]:
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
!pip install llama-index ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 69.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.2 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pre

In [4]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_API")

In [6]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts.prompts import SimpleInputPrompt
from llama_index.llms.huggingface import HuggingFaceLLM

SYSTEM_PROMPT = """Act as Muneer Raza's personal portfolio chatbot. 
Your task is to provide concised answer of question below about Muneer. 
Donot add false information, If you don't know answer, just say "I don't know". 
Provide direct answers without any introductory phrases or filler content, dont use 'based on context provided' or anything of introduction.
"""

query_wrapper_prompt = SimpleInputPrompt(
    "<|USER|>{query_str}<|ASSISTANT|>"
)

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Enable 8-bit quantization
    bnb_8bit_compute_dtype=torch.float32,  # Use a larger data type for computation
    bnb_8bit_quant_type="nf8",  # Specify the 8-bit quantization type
    bnb_8bit_use_double_quant=False,  # Typically, double quantization is not used with 8-bit
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=query_wrapper_prompt,
    system_prompt=SYSTEM_PROMPT,
    model_kwargs={"token":hf_token, "quantization_config":quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-base-en-v1.5"

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:

!wget 'https://raw.githubusercontent.com/MuneerRaza/Skill_Chat/main/VectorDB%20source.txt'

--2024-04-24 20:39:52--  https://raw.githubusercontent.com/MuneerRaza/Skill_Chat/main/VectorDB%20source.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5088 (5.0K) [text/plain]
Saving to: 'VectorDB source.txt'

VectorDB source.txt 100%[===================>]   4.97K  --.-KB/s    in 0s      

2024-04-24 20:39:53 (38.8 MB/s) - 'VectorDB source.txt' saved [5088/5088]



In [9]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader("/kaggle/working/").load_data()

In [10]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
# from llama_index.core.response.notebook_utils import display_response
# from llama_index.core.retrievers import VectorIndexRetriever
# from llama_index.core.query_engine import RetrieverQueryEngine
# from llama_index.core.postprocessor import SimilarityPostprocessor

# retriver=VectorIndexRetriever(index=vector_index, similarity_top_k=4)
# # postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

# # query_engine=RetrieverQueryEngine(retriever=retriver, node_postprocessors=[postprocessor])
# query_engine=RetrieverQueryEngine(retriever=retriver)

In [12]:
query_engine = vector_index.as_query_engine(streaming=True)

In [19]:
starter = 'Tell me very concisely: '
query = "List his AI projects"

resp = query_engine.query(starter+query)
resp.print_response_stream()
        

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Muneer Raza has developed and worked on the following AI projects:

1. Skill Chat Assistant
2. Facial Emotion Analysis
3. Aquatic Vision Enhancement
4. Mitotic Cell Detection
5. Biterm Text Distance Measurement (BDM)
6. Cifar-10 Image Classification
7. Image Segmentation
8. Resume Entity Recognition (NER)
9. Plant Disease Diagnosis
10. COVID-19 Radiological Analysis